# convert ntt_to_json

修改地址：dirname 进行测试

json Structure: 

{"ScNumber":,
  "CellNumber":, 
  "Params":, 
   "time_data" : {[Timestamp, Data],[Timestamp, Data],...,[Timestamp, Data]}
}

{"TSusec":, "Xpos1":, "Ypos1":, "Ang1":}

In [3]:
import os
import re
import json
from neo.io import NeuralynxIO

def convert_ntt_to_json(filename, input_dirname, output_dirname):

    # Ensure the output directory exists
    os.makedirs(output_dirname, exist_ok=True)

    nio = NeuralynxIO(dirname=input_dirname, filename=filename)
    block = nio.read_block()
    for seg in block.segments:
        tetrode_data = seg.spiketrains[0]
        timestamps = list(tetrode_data.times.rescale('s').magnitude)  # Convert to seconds

        cell_number_match = re.search(r'\d+', filename)
        cell_number = int(cell_number_match.group()) if cell_number_match else None

        # Construct "time_data" as specified

        data = {
            "CellNumber": cell_number,
            "Params": {
                "Sample Frequency (Hz)": float(tetrode_data.sampling_rate.rescale('Hz').magnitude),
                "Number of Valid Samples": int(tetrode_data.shape[0])
            },
            "Timestamps": timestamps
        }

        # Save to JSON file in the specified output directory
        json_filename = os.path.splitext(os.path.join(output_dirname, filename))[0] + '.json'
        with open(json_filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"Converted {filename} to {json_filename}")

# Example usage
input_dirname = r"/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1"
output_dirname = r"/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json"
ntt_file_list = [f for f in os.listdir(input_dirname) if f.endswith('.ntt')]

# Process each .NTT file
for ntt_file in ntt_file_list:
    convert_ntt_to_json(ntt_file, input_dirname, output_dirname)

Converted cut_TT9.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT9.json
Converted cut_TT8.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT8.json
Converted cut_TT10.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT10.json
Converted cut_TT11.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT11.json
Converted cut_TT15.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT15.json
Converted cut_TT5.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT5.json
Converted cut_TT6.ntt to /Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Dat

## Read json file example

In [4]:
import json

# Specify the path to the JSON file you want to read
json_filename = r"/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ntt_json/cut_TT2.json"

# Read the JSON file
with open(json_filename, 'r') as file:
    data = json.load(file)

feature_names = list(data.keys())
print("Feature Names:", feature_names)

# Display the base information for each key in the JSON file
for key, value in data.items():
    print(f"{key}:")
    if isinstance(value, list):
        # Print only the first few items if the value is a list
        print(value[:5])  # Modify this number based on how many examples you want to see
    else:
        print(value)

Feature Names: ['CellNumber', 'Params', 'Timestamps']
CellNumber:
2
Params:
{'Sample Frequency (Hz)': 32000.0, 'Number of Valid Samples': 237220}
Timestamps:
[0.01143700000102399, 0.023531000000730273, 0.024406000000453787, 0.028156000000308268, 0.032906000000366475]


# Convert ncs to json

修改地址：dirname 进行测试

'''
{
"CellNumber":,
"SampleFrequency":,
  "Number_of_valid_samples":, 
   "Time_data" : {[Timestamp, Samples],[Timestamp, Samples],...,[Timestamp, Samples]}
}
'''

In [2]:
import os
import json
import re
from neo.io import NeuralynxIO

def convert_ncs_to_json(filename, input_dirname, output_dirname):
    full_path = os.path.join(input_dirname, filename)
    reader = NeuralynxIO(filename=full_path)
    blk = reader.read_block()
    seg = blk.segments[0]
    signal = seg.analogsignals[0]

    timestamps = [(float(t), float(signal.magnitude[i])) for i, t in enumerate(signal.times.rescale('s').magnitude)]
    sample_frequency = float(signal.sampling_rate.rescale('Hz').magnitude)
    num_valid_samples = int(signal.shape[0])

    data_dict = {
        "CellNumber": re.search(r'CSC(\d+)_FSI20\.ncs', filename).group(1),  # Assuming 'CSC<Number>_FSI20.ncs' format
#         "CellNumber": re.search(r'\d+', filename),
        "SampleFrequency": sample_frequency,
        "Number_of_valid_samples": num_valid_samples,
        "Time_data": timestamps
    }

    json_data = json.dumps(data_dict, indent=4)
    json_filename = os.path.splitext(filename)[0] + '.json'
    json_file_path = os.path.join(output_dirname, json_filename)

    if not os.path.exists(output_dirname):
        os.makedirs(output_dirname)

    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)

    print(f"JSON file created: {json_filename}")

# Directory settings
input_dirname = "/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1"
output_dirname = "/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ncs_json"

# List all .ncs files
ncs_file_list = [f for f in os.listdir(input_dirname) if f.endswith('.ncs')]

# Process each .ncs file
for ncs_file in ncs_file_list:
    convert_ncs_to_json(ncs_file, input_dirname, output_dirname)

JSON file created: CSC6_FSI20.json
JSON file created: CSC15_FSI20.json
JSON file created: CSC13_FSI20.json
JSON file created: CSC7_FSI20.json
JSON file created: CSC1_FSI20.json
JSON file created: CSC4_FSI20.json
JSON file created: CSC9_FSI20.json
JSON file created: CSC2_FSI20.json
JSON file created: CSC11_FSI20.json
JSON file created: CSC5_FSI20.json
JSON file created: CSC16_FSI20.json
JSON file created: CSC8_FSI20.json
JSON file created: CSC10_FSI20.json


## Read json file example

In [3]:
import json

# Specify the path to the JSON file you want to read
json_filename = r"/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/ncs_json/CSC6_FSI20.json"

# Read the JSON file
with open(json_filename, 'r') as file:
    data = json.load(file)

feature_names = list(data.keys())
print("Feature Names:", feature_names)

# Display the base information for each key in the JSON file
for key, value in data.items():
    print(f"{key}:")
    if isinstance(value, list):
        # Print only the first few items if the value is a list
        print(value[:5])  # Modify this number based on how many examples you want to see
    else:
        print(value)

Feature Names: ['CellNumber', 'SampleFrequency', 'Number_of_valid_samples', 'Time_data']
CellNumber:
6
SampleFrequency:
1600.0
Number_of_valid_samples:
5579264
Time_data:
[[0.0, 84.53627014160156], [0.000625, 81.78960418701172], [0.00125, 81.05716705322266], [0.001875, 55.11642837524414], [0.0025, 41.07791519165039]]


In [5]:
# Specify the path to the JSON file you want to read
nvt_data = r"/Users/huangjuhua/NYU/MatlabFramework_SampleCode_Data/Tmaze_Data/252-1375/2018-01-07_15-14-54/04_tmaze1/nvt_data.json"

# Read the JSON file
with open(nvt_data, 'r') as file:
    data = json.load(file)

feature_names = list(data.keys())
print("Feature Names:", feature_names)

# Display the base information for each key in the JSON file
for key, value in data.items():
    print(f"{key}:")
    if isinstance(value, list):
        # Print only the first few items if the value is a list
        print(value[:5])  # Modify this number based on how many examples you want to see
    else:
        print(value)

Feature Names: ['TSusec', 'Xpos1', 'Ypos1', 'Ang1']
TSusec:
[9943225717.0, 9943259852.0, 9943292453.0, 9943326137.0, 9943359888.0]
Xpos1:
[30.294633785905607, 29.740463655675626, 29.740463655675626, 30.294633785905607, 29.925187032418954]
Ypos1:
[53.38505587882147, 52.646162371848156, 52.46143899510483, 52.830885748591484, 52.646162371848156]
Ang1:
[162, 160, 160, 166, 159]
